In [1]:
import numpy as np
import multiprocessing
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
from time import time
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns

sns.set_style("darkgrid")
%matplotlib inline

C:\Anaconda\envs\tf2\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


[Дообучение](https://github.com/sberbank-ai/ru-gpts/blob/master/examples/Finetune_RuGPTs_with_HF.ipynb) GPT3small на своем наборе данных

In [11]:
data=pd.read_csv('data.csv')
data.columns

Index(['date', 'sign', 'text'], dtype='object')

In [2]:
#signs=['aries','gemini','taurus','cancer','leo','virgo','libra','scorpio','sagittarius','capricorn','aquarius','pisces']
signs_r=['овен','близнецы','телец','рак','лев','дева','весы','скорпион','стрелец','козерог','водолей','рыбы']
sign_dict={'aries':'овен',
           'gemini':'близнецы',
           'taurus':'телец',
           'cancer':'рак',
           'leo':'лев',
           'virgo':'дева',
           'libra':' весы',
           'scorpio':'скорпион',
           'sagittarius':'стрелец',
           'capricorn':'козерог',
           'aquarius':'водолей',
           'pisces':'рыбы'}

In [13]:
data.values[0]

array(['2004-01-01', 'aries',
       'Любые разногласия во мнениях скоро улягутся, а вы продолжайте делать, как делали, но постарайтесь не наступать на ноги слишком многим, иначе ваши сегодняшние действия сыграют против вас в будущем.'],
      dtype=object)

In [22]:
with open("text.txt", "w", encoding="utf-8") as file:
    for line in data.values:
        sign=sign_dict[line[1]]
        text=line[2]
        
        context='<s>'+sign+'\n' +text+'</s>\n'
        
        file.write(context)
file.close()

In [19]:
max_val=0
for line in data.values[2]:
    if len(line)>max_val:
        max_val=len(line)
max_val

176

In [3]:
import numpy as np
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [4]:
tok = GPT2Tokenizer.from_pretrained("models/essays")
model = GPT2LMHeadModel.from_pretrained("models/essays")

In [5]:
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [7]:
for sig in signs_r:

    text = "<s> "+sig
    inpt = tok.encode(text, return_tensors="pt")
    out = model.generate(inpt.cuda(), 
                         max_length=40,
                         repetition_penalty=5.0, 
                         do_sample=True, top_k=5,
                         top_p=0.95,
                         temperature=1)
    predict=tok.decode(out[0])
    print(sig.capitalize()+'\n'+str(predict).capitalize()+'\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Овен
<s> овен
день благоприятен для различных контактов, визитов. повышается ваша деловая хватка и профессиональное честолюбие.</cъездовскаяпресссистема может оказать



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Близнецы
<s> близнецы
удачный день для близнецов. сегодня вы можете с лёгкостью реализовать свои задумки, если сумеете сочетать креативность и деловитость в главном направлении своей



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Стрелец
<s> стрелец
стрельцам не стоит откладывать разговор, консультацию или поездку. сегодня есть возможность договориться о встрече и обмене полезными услугами с нужными людьми: коллегами по работе либо родственниками



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Козерог
<s> козерог
у козерамов в этот день может появиться желание что-то изменить во внешности, поведении. звезды советуют вам не спешить с выводами относительно своих планов на ближайшее будущее



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Водолей
<s> водолей
в этот день водолеям стоит отказаться от инициатив. если вы хотите, чтобы вас заметили и запомнили на будущее (особенно за рубежом), вам предстоит проявить не только активность

Рыбы
<s> рыбы
сегодня вы можете с чистой совестью махнуть рукой на свои планы и заняться тем, к чему лежит душа. это подходящий момент для отдыха в компании друзей или любимого человека (особенно

